Some sources

https://github.com/librosa/librosa

https://magenta.tensorflow.org/onsets-frames

http://pythonforengineers.com/audio-and-digital-signal-processingdsp-in-python/

First generate a sin wave and save it as wav file http://pythonforengineers.com/audio-and-digital-signal-processingdsp-in-python/

In [3]:
import numpy as np
 
import wave
 
from struct import *
 
import matplotlib.pyplot as plt
 
# frequency is the number of times a wave repeats a second
 
frequency = 1000
 
num_samples = 48000
 
# The sampling rate of the analog to digital convert
 
sampling_rate = 48000.0
 
amplitude = 16000
 
file = "test.wav"

sine_wave = [np.sin(2 * np.pi * frequency * x/sampling_rate) for x in range(num_samples)]


nframes=num_samples
 
comptype="NONE"
 
compname="not compressed"
 
nchannels=1
 
sampwidth=2


wav_file=wave.open(file, 'w')
 
wav_file.setparams((nchannels, sampwidth, int(sampling_rate), nframes, comptype, compname))

for s in sine_wave:
   wav_file.writeframes(struct.pack('h', int(s*amplitude)))

In [ ]:
import pickle

In [1]:
print("hello")
a = [1,2,3,4,5]

hello


In [3]:
pickle.dump(a, open('test.p','wb'))

In [4]:
print("load pickle")

load pickle


In [5]:
loaded = pickle.load(file('test.p'))

In [6]:
print a

[1, 2, 3, 4, 5]
